# Overview

This document covers parsing an AIM format FSAE log. The data source can be found [here](https://huggingface.co/datasets/nominal-io/mission-operations-interview/tree/main)


In [4]:
# Import required packages and setup environment
import pandas as pd
import pandera.pandas as pa
import subprocess

filepath = "./data/raw/FSAE_Endurance_Full.csv"
parsed_filepath = "./data/processed/FSAE_Endurance_Full.csv"

In [5]:
# Extract the raw metadata (lines 1-13)
raw_metadata = {}
with open(filepath, "r", encoding="latin1") as f:
    for i in range(13):
        line = f.readline().strip().replace('"', "")
        left, right = line.split(",", 1)  # split on first comma
        raw_metadata[left] = right

# Process the metadata
metadata = {}
metadata["venue"] = raw_metadata["Venue"]
metadata["vehicle"] = raw_metadata["Vehicle"]
metadata["user"] = raw_metadata["User"]
metadata["data_source"] = raw_metadata["Data Source"]
metadata["comment"] = raw_metadata["Comment"]
metadata["start_time"] = pd.to_datetime(
    f"{raw_metadata['Date']} {raw_metadata['Time']}"
)
metadata["samples_per_sec"] = raw_metadata[
    "Sample Rate"
]  # This may be samples per second, or time between each sample
metadata["duration_sec"] = raw_metadata["Duration"]
metadata["segment"] = raw_metadata["Segment"]

beacon_markers_list = []
for item in raw_metadata["Beacon Markers"].split(","):
    beacon_markers_list.append(float(item.strip()))
metadata["beacon_markers_list"] = beacon_markers_list

segment_times_list = []
for item in raw_metadata["Segment Times"].split(","):
    item = item.strip()
    minutes, seconds = item.split(":")
    total_seconds = int(minutes) * 60 + float(seconds)
    segment_times_list.append(total_seconds)
metadata["segment_times_sec"] = segment_times_list

print(f"{'=' * 20} Metadata {'=' * 20}")
for item in metadata:
    print(f"{item}: {metadata[item]}")

==================== Metadata ====================
venue: FSAE2016_Endurance
vehicle: FSAE2016
user: Jon S. Giulio C.
data_source: AIM Data Logger
comment: 
start_time: 2016-05-14 16:27:26
samples_per_sec: 100
duration_sec: 1905.556
segment: Session
beacon_markers_list: [163.029, 240.928, 317.949, 396.714, 475.874, 554.335, 641.84, 718.689, 795.62, 880.075, 890.086, 1009.374, 1108.141, 1203.229, 1290.807, 1378.799, 1464.051, 1549.813, 1635.779, 1719.962, 1802.09, 1905.556]
segment_times_sec: [163.029, 77.899, 77.021, 78.765, 79.16, 78.461, 87.505, 76.849, 76.931, 84.455, 10.011, 119.288, 98.767, 95.088, 87.578, 87.992, 85.252, 85.762, 85.96600000000001, 84.18299999999999, 82.128, 103.46600000000001]


In [6]:
df = pd.read_csv(
    filepath,
    encoding="latin1",
    skiprows=15,  # Skip metadata, keep first column name row
    low_memory=False,  # Read entire file at once
)

df.columns = (
    df.columns.str.replace("_", ".") + "_" + df.iloc[0].astype(str)
)  # concat units to headers
df.columns = df.columns.str.lower()  # force headers to lowercase
df = df.drop(df.index[0]).iloc[
    2:
]  # Drop first row (appears to be an index of column ID)

# Type the dataframe using pandera
schema_field_dtypes = pa.DataFrameSchema(
    columns={
        ".*": pa.Column(float, regex=True, nullable=True),
    },
    index=pa.Index(int),
    strict=False,
    coerce=True,
)

# Validate the dataframe
df = schema_field_dtypes.validate(df)

# Rename columns
df.rename(columns={"time_sec": "time.lap_sec"}, inplace=True)

# Detect lap changes (when time_sec decreases)
df["lap"] = (df["time.lap_sec"] < df["time.lap_sec"].shift(1).fillna(0)).cumsum() + 1

# Calculate the maximum time from each previous lap
cumulative_offset = df.groupby("lap")["time.lap_sec"].max().shift(1).fillna(0).cumsum()

# Add the offset to current lap time
df["time.session_sec"] = df["lap"].map(cumulative_offset) + df["time.lap_sec"]

# Convert relative time to absolute time
df["time.absolute"] = metadata["start_time"] + pd.to_timedelta(
    df["time.session_sec"], unit="s"
)

# Save the parsed dataframe
df.to_csv(
    parsed_filepath,
    date_format="%Y-%m-%dT%H:%M:%S.%fZ",  # ISO 8601
    index=False,
)

# Print a pivoted head (many columns)
with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
    "display.width",
    None,
    "display.max_colwidth",
    None,
):
    print(f"{'=' * 20} Head {'=' * 20}")
    print(df.head(2).T)
    print(f"{'=' * 20} Tail {'=' * 20}")
    print(df.tail(2).T)

==================== Head ====================
                                                  3  \
time.lap_sec                                   0.01   
distance_km                                     0.0   
rr.shock_mm                               37.426758   
rl.shock_mm                               37.280273   
fl.shock_mm                                36.82251   
fr.shock_mm                               34.259033   
acc.lateral_g                              0.047369   
acc.longitudin_g                           -0.02596   
rear.brake_psi                             0.730953   
front.brake_psi                           -2.198718   
datalogger.tem_°f                         45.041924   
battery_v                                 13.437159   
f88.rpm_rpm                                     0.0   
f88.v.speed_mph                                 0.0   
f88.d.speed_mph                                 0.0   
f88.speed.fl_mph                                0.0   
f88.speed.fr_mph  

In [7]:
# quick sanity check to ensure we loaded the data correctly from the csv

lines_loaded = len(df)
lines_csv = int(subprocess.check_output(["wc", "-l", filepath], text=True).split()[0])

print(f"Lines in CSV file: {lines_csv}")
print(f"Lines loaded in DataFrame: {lines_loaded}")
print(f"Difference (metadata/header rows): {lines_csv - lines_loaded}")

Lines in CSV file: 190607
Lines loaded in DataFrame: 190587
Difference (metadata/header rows): 20
